**Inicialização**

In [1]:
# importando bibliotecas
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# fazendo a leitura do arquivo csv
try:
    df = pd.read_csv('Downloads\Churn.csv')
except:
    df = pd.read_csv('/datasets/Churn.csv')

**Analisando os dados**

In [3]:
# verificando as 5 primieras linhas do dataframe
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
# verificando as 5 ultimas linhas do dataframe
df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


In [5]:
# analisando informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
# verificando a quantidade de pessoas que ficaram e sairam
df['Exited'].value_counts(normalize = True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

**Tratamento dos dados / Pré-processamento**

Podemos observar colunas que não devem possuir relevancia para o treinamento do modelo:`RowNumber, CustomerId, Surname`.
Vamos remove-las! 

In [7]:
# eliminando colunas que não devem influenciar no treinamento
df = df.drop(['RowNumber','CustomerId','Surname'], axis = 1)

Vimos que existem valores nulos no nosso data frame na coluna `Tenure`, vamos tratar disso.

In [8]:
# verificando a média dos dados ausentes
df['Tenure'].isnull().mean()

0.0909

In [9]:
# substituindo os valores ausentes pela mediana
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())

In [10]:
# verifiando se a duplicatas
df.duplicated().sum()

0

In [11]:
# verificando as informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [12]:
# codificando todo o dataframe com OHE
df_ohe = pd.get_dummies(df, drop_first = True)

# verificando o tamanho da tabela 
df_ohe.shape

(10000, 12)

In [13]:
# verificando as 5 primieras linhas do novo dataframe
df_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


**Separando o conjuntos de dados**

In [14]:
# separando conjunto de caracteristicas do alvo
features = df_ohe.drop('Exited', axis=1)
target = df_ohe['Exited']

# dividir os dados em treinamento e teste
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# dividir os dados de treinamento em treinamento e validação
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)


**Escalabilidade das características**

In [15]:
# é utilizado para desabilitar um aviso que pode aparecer quando as variáveis 
# features_train, features_valid e features_test são modificadas.
pd.options.mode.chained_assignment = None

# separando as colunas numéricas
numeric = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

# criando uma instância da classe StandardScaler() para ajustar os dados
scaler = StandardScaler()

# ajustando o scaler apenas com os dados numéricos do conjunto de treinamento
scaler.fit(features_train[numeric])

# aplicando o scaler ajustado aos dados numéricos dos conjuntos de treinamento, validação e teste
# com o scaler aplicado, as colunas numéricas serão escalonadas para o intervalo de valores [-1, 1].
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

**Modelo - Regressão Logística**

In [16]:
# criação de um modelo de Regressão Logística
model_logistic_regression = LogisticRegression(random_state=12345, solver='liblinear')

# treinamento do modelo utilizando o conjunto de dados de treinamento
model_logistic_regression.fit(features_train, target_train)

# realização de predições no conjunto de dados de validação
valid_predicted = model_logistic_regression.predict(features_valid)

# cálculo das probabilidades para cada classe no conjunto de dados de validação
probabilities_valid = model_logistic_regression.predict_proba(features_valid)

# extração das probabilidades para a classe positiva (classe 1)
probabilities_valid_one = probabilities_valid[:,1]

# impressão das métricas de avaliação do modelo no conjunto de dados de validação
print('f1 score =',f1_score(target_valid,valid_predicted))
print('auc roc =', roc_auc_score(target_valid,probabilities_valid_one))

f1 score = 0.31640625
auc roc = 0.789968315437423


**Modelo - Árvore de Decisão**

In [17]:
# define variáveis iniciais
best_depth = 0
best_score_train = 0.0
best_score_valid = 0.0

# loop para avaliar a performance do modelo em diferentes profundidades
for depth in range(1, 15):
    
    # cria modelo com profundidade atual do loop
    model_decision_tree = DecisionTreeClassifier(random_state=12345,
                                                 max_depth=depth)
    
    # treinamento do modelo utilizando o conjunto de dados de treinamento
    model_decision_tree.fit(features_train, target_train)
    
    # realização de predições no conjunto de dados de validação
    predicted_valid = model_decision_tree.predict(features_valid)
    
    # cálculo das probabilidades para cada classe no conjunto de dados de validação
    probabilities_valid = model_decision_tree.predict_proba(features_valid)
    
    # extração das probabilidades para a classe positiva (classe 1)
    probabilities_valid_one = probabilities_valid[:,1]

    # avalia desempenho do modelo nos conjuntos de treinamento e validação
    score_f1 = f1_score(target_valid, predicted_valid)
    score_roc_auc = roc_auc_score(target_valid, probabilities_valid_one)

    # atualiza variáveis de melhor performance encontrada
    if score_f1 > best_score_valid:
        best_depth = depth
        best_score_roc_auc = score_roc_auc
        best_score_f1 = score_f1

# imprime melhor f1 e roc auc correspondentes
print(
    f"A melhor profundidade é {best_depth}\nf1 score de validação de {best_score_f1:.2f}\nroc_auc score de validação de {best_score_roc_auc:.2f}"
)

A melhor profundidade é 14
f1 score de validação de 0.54
roc_auc score de validação de 0.72


**Modelo - Floresta Aleatória**

In [18]:
# define variáveis iniciais
best_depth = 0
best_est = 0
best_score_train = 0.0
best_score_valid = 0.0
best_score_f1 = 0.0
best_score_roc_auc = 0.0

# loop externo para avaliar a performance do modelo em diferentes profundidades máximas
for depth in range(1, 20):

    # loop interno para avaliar a performance do modelo em diferentes número de estimadores
    for est in range(1, 10):

        # cria modelo com profundidade e número de estimadores atual do loop
        model_random_forest = RandomForestClassifier(random_state=12345,
                                                     n_estimators=est,
                                                     max_depth=depth)

        # treinamento do modelo utilizando o conjunto de dados de treinamento
        model_random_forest.fit(features_train, target_train)

        # realização de predições no conjunto de dados de validação
        predicted_valid = model_random_forest.predict(features_valid)

        # cálculo das probabilidades para cada classe no conjunto de dados de validação
        probabilities_valid = model_random_forest.predict_proba(features_valid)

        # extração das probabilidades para a classe positiva (classe 1)
        probabilities_valid_one = probabilities_valid[:, 1]

        # avalia desempenho do modelo nos conjuntos de treinamento e validação
        score_f1 = f1_score(target_valid, predicted_valid)
        score_roc_auc = roc_auc_score(target_valid, probabilities_valid_one)

        # verifica se o resultado atual é melhor do que o melhor encontrado até o momento
        if score_f1 > best_score_f1:
            best_depth = depth
            best_est = est
            best_score_train = model_random_forest.score(
                features_train, target_train)
            best_score_valid = model_random_forest.score(
                features_valid, target_valid)
            best_score_f1 = score_f1
            best_score_roc_auc = score_roc_auc

# imprime melhor f1 e roc auc correspondentes
print(
    f"Melhor profundidade: {best_depth}\nMelhor n_estimators: {best_est}\nF1 score de validação: {best_score_f1:.2f}\nROC AUC score de validação: {best_score_roc_auc:.2f}\n"
)

Melhor profundidade: 13
Melhor n_estimators: 9
F1 score de validação: 0.59
ROC AUC score de validação: 0.85



**Ajuste de ponderação da classe**

In [19]:
# cria modelo com profundidade e número de estimadores e usando class_weight
model_random_forest = RandomForestClassifier(random_state=12345,
                                             n_estimators=est,
                                             max_depth=depth,
                                             class_weight='balanced')

# treinamento do modelo utilizando o conjunto de dados de treinamento
model_random_forest.fit(features_train, target_train)

# realização de predições no conjunto de dados de validação
predicted_valid = model_random_forest.predict(features_valid)

# cálculo das probabilidades para cada classe no conjunto de dados de validação
probabilities_valid = model_random_forest.predict_proba(features_valid)

# extração das probabilidades para a classe positiva (classe 1)
probabilities_valid_one = probabilities_valid[:, 1]

# avalia desempenho do modelo nos conjuntos de treinamento e validação
score_f1 = f1_score(target_valid, predicted_valid)
score_roc_auc = roc_auc_score(target_valid, probabilities_valid_one)

# impressão das métricas de avaliação do modelo no conjunto de dados de validação
print('f1 score =', score_f1)
print('auc roc =', score_roc_auc)

f1 score = 0.5271317829457365
auc roc = 0.8216581587748636


**upsample**

In [20]:
# verificando classe majoritária e minoritária
df['Exited'].value_counts(normalize = True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

In [21]:
# define a função upsample que faz a reamostragem das observações da classe minoritária aumentando sua
# proporção em relação à classe majoritária. a função recebe três parâmetros: features (variáveis explicativas),
# target (variável target) e repeat (fator de aumento da classe minoritária).
def upsample(features, target, repeat):

    # cria quatro variáveis: features_zeros (observações da classe majoritária), features_ones
    # (observações da classe minoritária), target_zeros (alvo da classe majoritária), target_ones
    # (alvo da classe minoritária).
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    # concatena as features_zeros com features_ones repetidas repeat vezes e faz o mesmo para target_zeros e target_ones.
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    # embaralha as features e targets resultantes e retorna como a saída da função.
    features_upsampled, target_upsampled = shuffle(features_upsampled,
                                                   target_upsampled,
                                                   random_state=12345)

    # a função retorna duas variáveis: features_upsampled (variáveis explicativas reamostradas), target_upsampled
    #(variável target reamostrada)
    return features_upsampled, target_upsampled


# executa a função upsample com os parâmetros features_train, target_train e 10 e armazena as variáveis resultantes
# features_upsampled, target_upsampled.
features_upsampled, target_upsampled = upsample(features_train, target_train,
                                                10)

# imprime as dimensões das variáveis resultantes
print(features_upsampled.shape)
print(target_upsampled.shape)

(15723, 11)
(15723,)


**Treinando modelo usando o upsample**

In [22]:
# criando modelo
model_random_forest = RandomForestClassifier(random_state=12345,
                                             n_estimators=est,
                                             max_depth=depth)

# treinamento do modelo utilizando features_upsampled (variáveis explicativas reamostradas), target_upsampled
# (variável target reamostrada)
model_random_forest.fit(features_upsampled, target_upsampled)

# realização de predições no conjunto de dados de validação
predicted_valid = model_random_forest.predict(features_valid)

# cálculo das probabilidades para cada classe no conjunto de dados de validação
probabilities_valid = model_random_forest.predict_proba(features_valid)

# extração das probabilidades para a classe positiva (classe 1)
probabilities_valid_one = probabilities_valid[:, 1]

# avalia desempenho do modelo nos conjuntos de treinamento e validação
score_f1 = f1_score(target_valid, predicted_valid)
score_roc_auc = roc_auc_score(target_valid, probabilities_valid_one)

# impressão das métricas de avaliação do modelo no conjunto de dados de validação
print('f1 score =', score_f1)
print('auc roc =', score_roc_auc)

f1 score = 0.5879194630872483
auc roc = 0.8293548671008626


**Teste final**

In [23]:
# lista de valores para est e depth que desejamos testar de forma mais eficiente
est_values = [5, 10]
depth_values = [5, 10, 15, 20]

# define variáveis iniciais
best_score_f1 = 0
best_score_roc_auc = 0
best_est = 0
best_depth = 0

# loop aninhado para testar diferentes valores para est e depth
for est in est_values:
    for depth in depth_values:

        # concatenando os dados de treinamento e validação das features
        features_train_final = pd.concat([features_train] + [features_valid])

        # concatenando os dados de treinamento e validação do target
        target_train_final = pd.concat([target_train] + [target_valid])

        # criando o modelo de RandomForestClassifier com os valores de est e depth correspondentes
        # estabelecendo o parâmetro class_weight como "balanced"
        model_random_forest = RandomForestClassifier(random_state=12345,
                                                     n_estimators=est,
                                                     max_depth=depth,
                                                     class_weight='balanced')
        
        # treinamento do modelo utilizando features_upsampled (variáveis explicativas reamostradas), target_upsampled
        # (variável target reamostrada)
        model_random_forest.fit(features_upsampled, target_upsampled)
        
        # treinando o modelo com os dados de treinamento e validação concatenados
        model_random_forest.fit(features_train_final, target_train_final)

        # realizando previsões com os dados de teste
        predicted_test = model_random_forest.predict(features_test)

        # obtendo as probabilidades das previsões com os dados de teste
        probabilities_test = model_random_forest.predict_proba(features_test)

        # selecionando apenas as probabilidades da classe "1"
        probabilities_test_one = probabilities_test[:, 1]

        # calculando o score F1
        score_f1 = f1_score(target_test, predicted_test)

        # calculando o score ROC AUC
        score_roc_auc = roc_auc_score(target_test, probabilities_test_one)

        # verificando se essa combinação de est e depth apresentou um score melhor
        if score_f1 > best_score_f1:
            best_score_f1 = score_f1
            best_score_roc_auc = score_roc_auc
            best_est = est
            best_depth = depth

# imprimindo o melhor conjunto de parâmetros
print('Melhor est:', best_est)
print('Melhor depth:', best_depth)
print('Melhor f1 score:', best_score_f1)
print('Melhor roc auc score:', best_score_roc_auc)

Melhor est: 10
Melhor depth: 5
Melhor f1 score: 0.6279969064191803
Melhor roc auc score: 0.851155501652755


**Conclusão**

No início do projeto, realizamos o tratamento dos dados, pré-processamento, escalonamento das características e divisão dos conjuntos de dados. Em seguida, utilizamos três modelos: Regressão Logística, Árvore de Decisão e Floresta Aleatória. Com o objetivo de maximizar o desempenho do modelo, identificamos que a Floresta Aleatória apresentou o melhor resultado, alcançando aproximadamente **F1 score de validação de 0,59 e ROC AUC score de validação de 0,85**, já atingindo a meta estabelecida de *F1 score de pelo menos 0,59*. Dessa forma, decidimos utilizar o modelo de Floresta Aleatória para o restante do projeto.

Posteriormente, aplicamos o ajuste de ponderação da classe e treinamos o modelo usando o método upsample. No teste final, superamos nossa meta e obtivemos um resultado aproximado de **F1 score de 0,6279969064191803 e ROC AUC score de 0,851155501652755**